In [1]:
import plotly.offline as pyo
import plotly.graph_objs as go
import pandas as pd

A notebook to work on data prep and plotly visualizations so that I have things set up in a way that can be easily plugged in to the final dashboard. I'll be using plotly offline, so I'll simply insert static images screencapped from the html objects.

First a bit of data prep, using the city master csvs I prepared in the first big_deal notebook.

In [2]:
df = pd.read_csv('New_York_NY_master.csv')
df.head()

,years,TMAX,TMIN,t90,t95,t100
0,1947,46.821581,30.813200,0,0,0
1,1948,61.402767,42.641671,9,4,0
2,1949,64.488553,45.420937,20,8,0
3,1950,60.902360,43.011863,5,0,0
4,1951,62.462723,44.219303,1,0,0


A couple of quick prep steps - resetting the index, cutting off the first year (which is actually only a partial year's worth of data) and adding in trailing average columns, something I really should have done in my .py file that called the NOAA API and formatted these dataframes.

In [3]:
df.drop(0,inplace=True)
for metric in ['TMAX','TMIN','t90']:
    df[f'{metric}_rolling'] = [df[metric][0:n+1].mean() if n<10 else df[metric][n-10:n+1].mean() for
                              n in range(0,len(df))]
#Also need to datetime-ify the dates for later
df['years'] = pd.to_datetime(df['years'],format='%Y')    
df.set_index('years',inplace=True)
df.head()

,TMAX,TMIN,t90,t95,t100,TMAX_rolling,TMIN_rolling,t90_rolling
years,,,,,,,,
1948-01-01,61.402767,42.641671,9,4,0,61.402767,42.641671,9.000000
1949-01-01,64.488553,45.420937,20,8,0,62.945660,44.031304,14.500000
1950-01-01,60.902360,43.011863,5,0,0,62.264560,43.691490,11.333333
1951-01-01,62.462723,44.219303,1,0,0,62.314101,43.823443,8.750000
1952-01-01,62.634314,44.782412,12,1,0,62.378143,44.015237,9.400000


As of my beginning this notebook, I'm still a little unsure what I want the final visualizations to look like. Part of the goal here is exploratory and simply continuing to get a better feel for the plotly layout.

Part of what makes plotly so excellent is that the default styling, without a whole lot of extra work, already looks pretty good.

In [4]:
metric = 'TMAX'
data = [go.Scatter(x=df.index, y=df[column], mode='lines') for column in [metric,f'{metric}_rolling']]
layout = go.Layout(title = f'{metric} and 10 year rolling average')
fig = go.Figure(data = data, layout=layout)
pyo.plot(fig)

'temp-plot.html'

Produces:
![basic_no_formatting.png](https://github.com/max-miller/big_deal/blob/master/visualizations/basic_no_formatting.png?raw=true)

This does remind that at the very least I need to give the traces their appropriate names

In [5]:
metric = 'TMAX'
data = [go.Scatter(x=df.index, y=df[column], mode='lines', name=column) for column in [metric,f'{metric}_rolling']]
layout = go.Layout(title = f'{metric} and 10 year rolling average')
fig = go.Figure(data = data, layout=layout)
pyo.plot(fig)

'temp-plot.html'

![with_names.png](https://github.com/max-miller/big_deal/blob/master/visualizations/with_names.png?raw=true)

In [6]:
metric = 'TMIN'
data = [go.Scatter(x=df.index, y=df[column], mode='lines', name=column) for column in [metric,f'{metric}_rolling']]
layout = go.Layout(title = f'{metric} and 10 year rolling average')
fig = go.Figure(data = data, layout=layout)
pyo.plot(fig)

'temp-plot.html'

The most important thing for this project, interactivity wise, is the ability to select metric or city. For that I'm going to try using Dash.

I'm a little new to this. But the first simple thing is to set up Dash to take in a metric (TMAX, TMIN, t90) and return the appropriate graph. 

In [7]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [8]:
app= dash.Dash() #create Dash app
#layout for now is simple, just the graph and the dropdown with metrics.
app.layout = html.Div([
                dcc.Graph(id='graph'),
                dcc.Dropdown(id='metric-dropdown',
                            options=[{'label':'Average Maximum Temp','value':'TMAX'},
                                    {'label':'Average Minimum Temp','value':'TMIN'},
                                    {'label':'Number of days above 90','value':'t90'}])])

#callback is what actually links the input from the dropdown to some other component, here I specify
#that the output will be the component with the id 'graph'
@app.callback(Output('graph','figure'),
             [Input('metric-dropdown','value')])

#This function creates the appropriate plotly style plot from the inputted metric.
def update_figure(metric):
    data = [go.Scatter(x=df.index, y=df[column], mode='lines', name=column) 
            for column in [metric,f'{metric}_rolling']]
    layout = go.Layout(title = f'{metric} and 10 year rolling average')
    return {'data':data,'layout':layout}


app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


Having never made interactive visualizations like this before, I'm very happy with how easy it was to create these, and it's clear to me how to extend this to encompass a bunch of other things I'll want to do.

![TMAX](https://github.com/max-miller/big_deal/blob/master/visualizations/tmax_dash.png?raw=true)

![TMIN](https://github.com/max-miller/big_deal/blob/master/visualizations/tmin_dash.png?raw=true)

One quirk I need to figure out is how to get it to default to a value, since currently, when I start it up, there is no default graph populated as it waits for the user to input a metric:

![TMIN](https://github.com/max-miller/big_deal/blob/master/visualizations/empty_graph.png?raw=true)


Looking ahead, there are a couple of content things to work on and then some styling things. I'm going to save the styling for last, since that is what I'm least familiar with. For now I'm just going to make sure that everything I want is on the page, then I'll go back and make sure it looks pretty.

1. Unify all the labels with a master dictionary
2. Create a function to perform a two sample ttest 
3. Dynamicall incorporate results of ttest onto dashboard

Ttest function will perform the statisitical test and return the appropriate text describing the results:

In [9]:
label_dict = {'TMAX':'Average Daily Maximum Temperature','TMIN':'Average Daily Minimum Temperature',
             't90':'Number of Days Above 90 Degrees','TMAX_rolling':'Rolling average',
              'TMIN_rolling':'Rolling average','t90_rolling':'Rolling average'}

from scipy import stats

def ttest(df, metric):
    diff = round(df['2009':'2018'][metric].mean()-df['1950':'2000'][metric].mean(),2)
    pvalue = stats.ttest_ind(df['2009':'2018'][metric],df['1950':'2000'][metric],equal_var=False)[1]
    if pvalue == 0:
        if diff >0:
            return f'''The {label_dict[metric].lower()} was {diff} higher in the last ten years compared to last half of the 20th century. The odds of this happening by chance are vanishingly small'''
        return f'''The {label_dict[metric].lower()} was {-diff} lower in the last ten years compared to last half of the 20th century. The odds of this happening by chance are vanishingly small'''
    elif pvalue < .2:
        odds = int(round(1/pvalue))
        if diff >0:
            return f'''The {label_dict[metric].lower()} was {diff} higher in the last ten years compared to last half of the 20th century. The odds of this happening by chance are about 1 in {odds}'''
        
        return f'''The {label_dict[metric].lower()} was {-diff} lower in the last ten years compared to last half of the 20th century. The odds of this happening by chance are about 1 in {odds}'''
    elif diff >0:
        return f'''The {label_dict[metric].lower()} was {diff} higher in the last ten years compared to last half of the 20th century. This difference may well be due to chance'''
    return f'''The {label_dict[metric].lower()} was {-diff} lower in the last ten years compared to last half of the 20th century. This difference may well be due to chance'''

In [10]:
ttest(df,'TMAX')

'The average daily maximum temperature was 1.25 higher in the last ten years compared to last half of the 20th century. The odds of this happening by chance are about 1 in 66'

First I just brought the two callbacks in simply to test it out. Only styling I did was to put the dropdown menu at the top. It was also a simple to fix the default value for the dropdown list:

In [11]:
app= dash.Dash()

app.layout = html.Div([dcc.Dropdown(id='metric-dropdown',
                            options=[{'label':'Average Maximum Temp','value':'TMAX'},
                                    {'label':'Average Minimum Temp','value':'TMIN'},
                                    {'label':'Number of days above 90','value':'t90'}],
                            value='TMAX'),
                       dcc.Graph(id='graph'),
                       html.Div(id='ttest')
                        ])

@app.callback(Output('graph','figure'),
             [Input('metric-dropdown','value')])


def update_figure(metric):
    data = [go.Scatter(x=df.index, y=df[column], mode='lines', name=label_dict[column]) 
            for column in [metric,f'{metric}_rolling']]
    layout = go.Layout(title = f'{metric} and 10 year rolling average')
    return {'data':data,'layout':layout}

@app.callback(Output('ttest','children'),
             [Input('metric-dropdown','value')])
def perform_ttest(metric):
    return(ttest(df,metric))



app.run_server()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Aug/2019 14:32:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 14:32:33] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1561062262 HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 14:32:33] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1561062262 HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 14:32:33] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1561062262 HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 14:32:33] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1561063963 HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 14:32:33] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1563307603 HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 14:32:33] "GET /_dash-component-suites/dash_renderer/dash_renderer.min.js?v=1.0.0&m=1561062262 HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 1

Simple results are pretty good:

![dash_with_ttest](https://github.com/max-miller/big_deal/blob/master/visualizations/dash_with_ttest.png?raw=true)

Next took a bunch of trial/error to get the styling to align the text and the graphs the way I wanted and to move the legend to the bottom and properly align it.

In [12]:
app= dash.Dash()

app.layout = html.Div([dcc.Dropdown(id='metric-dropdown',
                            options=[{'label':'Average Maximum Temp','value':'TMAX'},
                                    {'label':'Average Minimum Temp','value':'TMIN'},
                                    {'label':'Number of days above 90','value':'t90'}],
                            value='TMAX'),
                       html.Div([html.Div(dcc.Graph(id='graph'),style={'width':'75%','display':'inline-block',
                                                                      'vertical-align': 'middle'}),
                                html.Div(id='ttest',style={'width':'25%','display':'inline-block',
                                                          'vertical-align': 'middle',
                                                          'fontSize':20})])
                        ])

@app.callback(Output('graph','figure'),
             [Input('metric-dropdown','value')])


def update_figure(metric):
    data = [go.Scatter(x=df.index, y=df[column], mode='lines', name=label_dict[column]) 
            for column in [metric,f'{metric}_rolling']]
    layout = go.Layout(title = f'{metric} and 10 year rolling average',
                       legend=go.layout.Legend(xanchor='center', x=0.5, orientation='h'))
    return {'data':data,'layout':layout}

@app.callback(Output('ttest','children'),
             [Input('metric-dropdown','value')])
def perform_ttest(metric):
    return(ttest(df,metric))

app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Aug/2019 14:35:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 14:35:41] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1561062262 HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 14:35:41] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1561062262 HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 14:35:41] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1561062262 HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 14:35:41] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1561063963 HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 14:35:41] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1563307603 HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 14:35:41] "GET /_dash-component-suites/dash_renderer/dash_renderer.min.js?v=1.0.0&m=1561062262 HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 1

![aligned_ttest](https://github.com/max-miller/big_deal/blob/master/visualizations/dash_aligned_ttest.png?raw=true)